In [1]:
import pandas as pd
import pickle
from collections import defaultdict
from DeepRGARCH.rerech import model as mdl
from rerech import utils as ut

ModuleNotFoundError: No module named 'DeepRGARCH'

In [3]:
data_dict = ut.build_data_dict(ut.load_rv(filepath='/Users/ananyaparikh/Documents/Coding/DeepRGARCH/code submission/data/rv.pkl'))
tickers = list(data_dict.keys())
model_dict = defaultdict(lambda: {})

/Users/ananyaparikh/Documents/Coding/DeepRGARCH/DeepRGARCH/rerech/utils.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['rv5'][df['rv5']<=0] = df['rv5'][df['rv5']>0].min()
/Users/ananyaparikh/Documents/Coding/DeepRGARCH/DeepRGARCH/re

In [4]:
for ticker in tickers:
    data = data_dict[ticker]['train']
    y_train, y_test, rv_train, rv_test = data['y_train'], data['y_test'], data['rv_train'], data['rv_test']
    
    garch = mdl.GARCH(prior=ut.garch_prior,cstr_fn=ut.garch_cstr_fn,data=y_train)
    garch.run()
    model_dict[ticker]['garch'] = mdl.GARCHD(pre=garch, data=[y_train, y_test])
    model_dict[ticker]['garch'].run() 
    with open('checkpoint/{}_{}_{}.pkl'.format(ticker, 'garch', n), 'wb') as f:
        pickle.dump(model_dict[ticker]['garch'], f)

    rech = mdl.RECH(prior=ut.rech_prior,data=y_train)
    rech.run() 
    model_dict[ticker]['rech'] = mdl.RECHD(pre=rech, data=[y_train, y_test])
    model_dict[ticker]['rech'].run()
    with open('checkpoint/{}_{}_{}.pkl'.format(ticker, 'rech', n), 'wb') as f:
        pickle.dump(model_dict[ticker]['rech'], f)
    
    realgarch = mdl.RealGARCH(prior=ut.realgarch_prior,data=[y_train, rv_train])
    realgarch.run()
    model_dict[ticker]['realgarch'] = mdl.RealGARCHD(pre=realgarch, data=[y_train, y_test, rv_train, rv_test])
    model_dict[ticker]['realgarch'].run()
    with open('checkpoint/{}_{}_{}.pkl'.format(ticker, 'realgarch', n), 'wb') as f:
        pickle.dump(model_dict[ticker]['realgarch'], f)

    realrech = mdl.RealRECH(prior=ut.realrech_prior,data=[y_train, rv_train])
    realrech.run()
    model_dict[ticker]['realrech'] = mdl.RealRECHD(pre=realrech, data=[y_train, y_test, rv_train, rv_test])
    model_dict[ticker]['realrech'].run()
    with open('checkpoint/{}_{}_{}.pkl'.format(ticker, 'realrech', n), 'wb') as f:
        pickle.dump(model_dict[ticker]['realrech'], f)

/Users/ananyaparikh/Documents/Coding/DeepRGARCH/DeepRGARCH/rerech/model.py:20: RuntimeWarning: overflow encountered in multiply
  var[n+1] = theta['omega'] + theta['alpha'] * np.square(self.Y[n]) + theta['beta'] * var[n]
/Users/ananyaparikh/Documents/Coding/DeepRGARCH/DeepRGARCH/rerech/model.py:21: RuntimeWarning: invalid value encountered in sqrt
  llik = np.sum(stats.norm.logpdf(self.Y[1:], loc=0, scale = np.sqrt(var[1:])), axis=0) #y0, var0 is excluded, end with yt, vart
/Users/ananyaparikh/Documents/Coding/DeepRGARCH/DeepRGARCH/rerech/model.py:44: RuntimeWarning: invalid value encountered in sqrt
  llik = np.sum(stats.norm.logpdf(Y, loc=0, scale = np.sqrt(var)), axis=0)


NameError: name 'n' is not defined